# Element Parameter Detection

## Setup

In [ ]:
import numpy as np
import math
import random
import os
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance

import ifcopenshell
from path import Path
import open3d as o3d

from src.elements import *
from src.ifc import *
from src. preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance
from src.icp import icp_finetuning
from src.chamfer import *
from src.utils import *
from src.plots import plot_error_graph, plot_parameter_errors
from src.pca import testset_PCA
from src.finetune import chamfer_fine_tune
from tqdm.notebook import tqdm_notebook as tqdm


In [ ]:
random.seed = 42

In [ ]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'

## Model

## Test

Analyze results statistically

POINTNET++

In [ ]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
inference = False
if inference:
    path = Path('output/bp_data/')
    ext = ".ply"
else:
    path = Path('output/')
    ext = ".pcd"

cat= 'elbow'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False
cat_targets = {"elbow":14, "tee":19, "pipe":11}

if inference:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms, inference=True)
    targets = cat_targets[cat]
else:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
    targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=32)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = "pointnet2/log/classification/pointnet2_cls_ssg/"
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

checkpoint = torch.load(model_path + '/checkpoints/best_model.pth')
#checkpoint = torch.load(model_path + '/checkpoints/models/best_model_t_chamfer_0005.pth')
predictor.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
blueprint = 'data/sample.ifc'
temp_dir = "output/temp/"
target_dir = "output/tee/test/"

ifcConvert_executable = "scripts/./IfcConvert"
cloudCompare_executable = "cloudcompare.CloudCompare"
sample_size = 2048
threshold = 2

In [ ]:
def model_inference(model, loader, device, calculate_score=False):
    predictor = model.eval()
    predictions_list, pcd_list, id_list = [], [], []
    with torch.no_grad():
        for j, data  in tqdm(enumerate(loader), total=len(loader)):
            points, ids = data['pointcloud'].to(device).float(), data['id'].to(device)
            points = points.transpose(2, 1)
            preds, _ = predictor(points)
            preds, points, ids = preds.to(torch.device('cpu')), points.to(torch.device('cpu')), data['id'].to(torch.device('cpu'))
            for i, pr in enumerate(preds):
                predictions_list.append(pr.numpy())
                pcd_list.append(points[i].numpy())
                id_list.append(ids[i].numpy())

        return (predictions_list, pcd_list, id_list)

In [ ]:
if inference:
    predictions_list, cloud_list, id_list = model_inference(predictor.eval(), testDataLoader, device)

In [ ]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 0
    tot = 0
    count = 0
    
    for j, data  in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points, target, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device('cpu'))
        inputs = points.to(torch.device('cpu'))
        labels = target.to(torch.device('cpu'))
        ids = ids.to(torch.device('cpu'))
        #print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            #print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)        

    return predictions_list, inputs_list, label_list, output_list, id_list, cloud_list

In [ ]:
if not inference:
    with torch.no_grad():
        predictions_list, inputs_list, label_list, output_list, id_list, cloud_list = test(predictor.eval(), testDataLoader, device, test_criterion)

    print(len(predictions_list), len(inputs_list))

In [ ]:
# k = 5
# j = 4
# direction = False
# use_direction = True
# square_error = True
# error_calc(predictions_list, inputs_list, k, j, direction, use_direction, square_error)

In [ ]:
if not inference:
    label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

In [ ]:
# ratio = np.absolute((label_list - output_list)/label_list)
# ratio_ind = ratio.argsort()
# id_list = id_list[ratio_ind]
# print(id_list[-10:-1])

# error_threshold = 0.1
# correct = ratio[np.where(ratio < error_threshold)]
# print(len(ratio), len(correct), len(correct)/len(ratio))


In [ ]:
# cloud_id = 2
# pcd_id = 24229  


#### Visually analyse predictions and Fine tune with ICP, calculate chamfer distances

In [ ]:
# scaling up and down is required for icp calculations
def chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    error_count = 0

    # get predictions and pcds
    for i in tqdm(range(len(predictions_list))):
    #for i in tqdm(range(50)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)

        try:
            if  icp_correction:
                # note: preds are updated in place during ICP
                _, _ = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                     ifcConvert_executable, cloudCompare_executable, sample_size, threshold, False)

            preds_list.append(preds)
            pcd_list.append(pcd)

        except Exception as e:
            print("ICP error", pcd_id, e)
            error_count += 1

    # calculate chamfer distances
    cuda = torch.device('cuda')    
    rescaled_preds = [scale_preds(preds, cat, up=0) for preds in preds_list]
    preds_t = torch.tensor(rescaled_preds, requires_grad=True, device=cuda)
    cloud_t = torch.tensor(cloud_list, device=cuda)
    
    chamfer_dists = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False)
    chamfer_dists = chamfer_dists.detach().cpu().numpy()
    
#     for i, preds in enumerate(tqdm(preds_list)):
#         preds = scale_preds(preds, cat, up=0)
#         chamfer_distance, _ = get_chamfer_dist_single(pcd_list[i], preds, cat)
#         chamfer_dists.append(chamfer_distance)
        

    
    print("error_count", error_count)
    return chamfer_dists

In [ ]:
dists = chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
if inference:
    with open(model_path + 'preds_' + cat + '.pkl', 'wb') as f:
        pickle.dump([predictions_list, id_list, dists], f)
    
plot_error_graph(dists, "chamfer loss")

In [ ]:
# scaling up and down is required for icp calculations
def visualise__predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    viewer_list, ifc_list = [], []
    error_count = 0

    # get predictions and pcds
    #for i in tqdm(range(len(predictions_list))):
    for i in tqdm(range(20)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0).tolist(), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        print(pcd[0])
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, ext)

        try:
            if  icp_correction:
                # note: preds are updated in place during ICP
                viewer, ifc = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                     ifcConvert_executable, cloudCompare_executable, sample_size, threshold, True)
            else:
                #print(preds)
                viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)


            preds_list.append(preds)
            pcd_list.append(pcd)
            viewer_list.append(viewer)
            ifc_list.append(ifc)

        except Exception as e:
            print("ICP error", pcd_id, e)
            error_count += 1


    print("error_count", error_count)    
    return viewer_list

In [ ]:
viewers = visualise__predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
for v in viewers:
    print(v)

In [ ]:
# icp_correction = False
# viewers, ifcs = [], []
# preds_list, pcd_list = [], []

# for i in range(1,4):
#     pcd_id = id_list[i].item()
#     #print(pcd_id) 
        
#     #print("p", predictions_list[i], "in", inputs_list[i])

# #     #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)
# #     pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
# #     print(pcd.shape)
# #     preds = scale_preds(preds.tolist(), cat)
# # #     pcd2, input_preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, path, ext)
# # #     indices_to_replace = [5, 6, 7]
# # #     for i in indices_to_replace:
# # #         preds[i] = input_preds[i]
    
# #     #try:
# #     if not icp_correction:
# #         viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)

# #     else:
# #         viewer, ifc = icp_finetuning(pcd, pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
# #                              ifcConvert_executable, cloudCompare_executable, sample_size, threshold)

# #     viewers.append(viewer) 
# #     ifcs.append(ifc)
# #     preds_list.append(preds)
# #     pcd_list.append(pcd)
       
# # #     except Exception as e:
# # #         print("ICP error", pcd_id, e)
        
#     preds_list.append(inputs_list[i])
#     pcd_list.append(cloud_list[i].transpose(1, 0))

# cloud = o3d.geometry.PointCloud()
# chamfer_dists = []
# for i, preds in enumerate(preds_list):
#     pcd = pcd_list[i]
#     chamfer_distance, cl = get_chamfer_dist_single(pcd, preds, cat)
#     chamfer_dists.append(chamfer_distance)
    
# print("chamfer_distance", sum(chamfer_dists)/len(chamfer_dists))
# print("chamfer_distance", chamfer_dists)
# cloud.points = o3d.utility.Vector3dVector(np.array(cl))
# o3d.io.write_point_cloud("generated.pcd", cloud)
# cloud.points = o3d.utility.Vector3dVector(pcd)
# o3d.io.write_point_cloud("pcd.pcd", cloud)


# # #tensor based chamfer dist
# # preds_tensor = torch.tensor(preds_list).cuda()
# # pcd_tensor = torch.transpose(torch.tensor(pcd_list).cuda().float(), 1,2)
# # #print(pcd_tensor.shape)
# # chamfer_distance = get_chamfer_loss_tensor(preds_tensor, pcd_tensor)
# # print("chamfer", chamfer_distance)


# # for i, cl in enumerate(clouds_tensor.detach().cpu().numpy()):
# #     cloud.points = o3d.utility.Vector3dVector(cl)
# #     o3d.io.write_point_cloud("axis_"+str(i)+".pcd", cloud)
# #     cloud.points = pcd_list[i]
# #     o3d.io.write_point_cloud("pcd_"+str(i)+".pcd", cloud)
    
# # cloud = generate_elbow_cloud(preds)
    
# #chamfer_distance = get_chamfer_dist_single(cloud, pcd)
# #print("chamfer_distance", chamfer_distance)




In [ ]:
plot_parameter_errors(inputs_list, predictions_list, cat)

#### BP data Visualisation

In [ ]:
#batch_visualise(model_path, blueprint, path, ext, device, ifc=False)

In [ ]:
#merge_clouds(path, 'pipe')

### Misc

In [ ]:
# PCA
#testset_PCA(cloud_list, inputs_list, testDataLoader, cat)

In [ ]:
print(torch. __version__)

In [ ]:
# preds = inputs_list[:512]
# #preds = scale_preds(preds, cat, up=0)
# pcd = cloud_list[1].transpose(1, 0)
# print(preds)

# preds_tensor = torch.tensor(np.array(preds)).cuda().float()
# gen_cloud = generate_tee_cloud_tensor(preds_tensor)
# print(gen_cloud.shape)
# #gen_cloud = generate_pipe_cloud(preds)

# points = gen_cloud.detach().cpu().numpy()
# #points = np.array(gen_cloud)
# cloud = o3d.geometry.PointCloud()
# print(type(points), points.shape)
# cloud.points = o3d.utility.Vector3dVector(points[0])
# #cloud.points = o3d.utility.Vector3dVector(points)
# o3d.io.write_point_cloud("pipe_gen.pcd", cloud)

# cloud.points = o3d.utility.Vector3dVector(pcd)
# o3d.io.write_point_cloud("pipe_input.pcd", cloud)

In [ ]:
# chamfer loss
#v, loss = chamfer_fine_tune(50, 0.0001, predictions_list[0], cloud_list[0], cat, blueprint)

In [ ]:
limit = 20
v, modified_preds = chamfer_fine_tune(50, 0.01, predictions_list[:limit], cloud_list[:limit], cat, blueprint, alpha=3)
#v = thread_wrapper(50, 0.0001, predictions_list[:5], cloud_list[:5], cat)

In [ ]:
print(v)

In [ ]:
dists = chamfer_evaluate(modified_preds, cloud_list[:limit], id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

plot_error_graph(dists, "chamfer loss")

In [ ]:
# test gradient descent on random predictions
print(type(predictions_list[0][0]))
rand_preds = np.random.rand(predictions_list[0].shape[0]).astype(np.float32)
v, modified_preds = chamfer_fine_tune(50, 0.05, [rand_preds], [cloud_list[0]], cat, blueprint)

In [ ]:
v

## Mesh deformation